In [1]:
import os
import random

import pandas as pd
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transformss
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [2]:
# model, garment, label = df.iloc[10]
# image = Image.open(os.path.join("data/DressCode", LABEL_MAP[label], "images", garment))
# image.convert('RGB')
# colors = image.getcolors(image.size[0] * image.size[1])
# colors
# # Diplay colors of image
# most_common = max(colors, key=lambda x: x[0])
# most_common[1]


In [3]:
LABEL_MAP = [
    "upper_body",
    "lower_body",
    "dresses"
]

class FashionDataset(Dataset):
    def __init__(self, root: str, pairs: str) -> None:
        super().__init__()
        
        self.root = root

        # Load in the paired data
        self.data = pd.read_csv(pairs, delimiter='\t', header=None, names=['model', 'garment', 'label'])

    def __len__(self) -> int:
              return len(self.data)
    
    def __getitem__(self, index: int) -> dict:
        model, garment, label = self.data.iloc[index]
        
        # Load the images
        anchor = Image.open(os.path.join(self.root, LABEL_MAP[label], "images", garment)).convert("RGB")
        positive = Image.open(os.path.join(self.root, LABEL_MAP[label], "images", model)).convert("RGB")
        
        # Load the negative image
        negative_index = random.randrange(0, len(self.data))
        negative_model, negative_garment, negative_label = self.data.iloc[negative_index]
        negative = Image.open(os.path.join(self.root, LABEL_MAP[negative_label], "images", negative_model)).convert("RGB")

        # Convert images to tensors
        convert_tensor = transformss.ToTensor()
        anchor = convert_tensor(anchor)
        positive = convert_tensor(positive)
        negative = convert_tensor(negative)
        
        return anchor, positive, negative

In [4]:
def train(model: nn.Module, train_data: DataLoader, test_data: DataLoader, loss_fcn: nn.Module, epochs: int = 10, device: str = "cpu"):

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Tensorboard logger
    logger = SummaryWriter("./logs/Landmark V1.0")

    for epoch in range(epochs):

        # Set model to training mode
        model.train()

        for i, (anchor, positive, negative) in tqdm(enumerate(train_data), f"Epoch {epoch}", unit="batch"):
            optimizer.zero_grad()

            # Send images to the device
            anchor = anchor.to(device)
            positive = positive.to(device)
            negative = negative.to(device)

            # Forward pass
            anchor_features = model(anchor)
            positive_features = model(positive)
            negative_features = model(negative)

            # Compute the loss
            loss = loss_fcn(anchor_features, positive_features, negative_features)

            # Log loss to tensorboard
            logger.add_scalar("Triplet Loss", loss, i)

            # Backward pass
            loss.backward()
            optimizer.step()

        # Evaluate the model on the testing data
        model.eval()

        validation_loss = 0.0

        with torch.no_grad():
            for i, (anchor, positive, negative) in tqdm(enumerate(test_data), "Testing", unit="batch"):
                anchor.to(model.device)
                positive.to(model.device)
                negative.to(model.device)

                anchor_features = model(anchor)
                positive_features = model(positive)
                negative_features = model(negative)

                #TODO: Add some more metrics to the validation loop (i.e. euclidean distance between a & p, a & n, etc...)
                
                # Compute the loss
                loss = loss_fcn(anchor_features, positive_features, negative_features)

                validation_loss += loss

        logger.add_scalar("Validation Loss", validation_loss / len(test_data), epoch)

        print(f"Epoch {epoch} Validation Loss: {validation_loss / len(test_data)}")

In [5]:
# Load the dataset
train_data = FashionDataset("data/DressCode", "data/DressCode/train_pairs.txt")

test_data = FashionDataset("data/DressCode", "data/DressCode/test_pairs_paired.txt")


# Define the training dataloader
train_loader = DataLoader(train_data, batch_size=4, shuffle=True)

# Define the validation dataloader
test_loader = DataLoader(test_data, batch_size=4, shuffle=False)

In [6]:
# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Set the seed
torch.manual_seed(42)

# Load the model
model = models.resnet50()

model = model.to(device)

In [7]:
train(model, train_loader, test_loader, nn.TripletMarginLoss(), epochs=100, device=device)

Epoch 0: 37batch [03:26,  5.57s/batch]


KeyboardInterrupt: 